In [8]:
import numpy as np
import seaborn as sns
import sys
sys.path.append('/home3/ebrahim2/beyond-brainscore/analyze_results/figures_code/')
from trained_untrained_results_funcs import find_best_layer, loop_through_datasets
# include punctuation
from scipy.ndimage import gaussian_filter1d

In [9]:
%config InlineBackend.figure_format='svg'
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Helvetica'

### Pereira 

In [22]:
from copy import deepcopy
sigma_range = np.round(np.arange(0, 4.8, 0.1),2)
# load the positional and word rate information 
positional_WN = np.load('/data/LLMs/data_processed/pereira/acts/X_positional_WN.npz')['layer1']
psg_start = np.where(positional_WN[:,0] == 1)
# Add the final index to make the following for loop work
psg_start = np.insert(psg_start[0], 168, 627)
positional_WN_dict = {}
for s in sigma_range:
    positional_WN_dict[f'layer_{s}'] = deepcopy(positional_WN)
    if s!=0:
        for i in range(len(psg_start)-1):
                positional_WN_dict[f'layer_{s}'][ psg_start[i] : psg_start[i+1] , 0 : psg_start[i+1] -psg_start[i]] =  gaussian_filter1d(
                positional_WN[ psg_start[i] : psg_start[i+1] , 0 : psg_start[i+1] -psg_start[i]], sigma=s) 
                
np.savez('/data/LLMs/data_processed/pereira/acts/X_positional_WN_smooth.npz', **positional_WN_dict)

### Fedorenko

In [5]:


word_len = []
with open('/data/LLMs/data_processed/fedorenko/text/sentences_ordered.txt', 'r') as file:
    for line in file:
        words = line.split()
        word_len.extend([len(word) for word in words])
    
word_len = np.expand_dims(word_len, axis=-1)
growing_act = np.expand_dims(np.tile([0, 1, 2, 3, 4, 5, 6, 7], 52),axis=-1)

# Initialize the matrix
rows, cols = 416, 8
pos_matrix = np.zeros((rows, cols))

# Fill the matrix with the cyclic one-hot encoding
for i in range(rows):
    pos_matrix[i, i % cols] = 1
    
sigma_range = np.round(np.arange(0, 4.8, 0.1),2)
position_fed = {}
wn_position_fed = {}
for s in sigma_range:
    if s == 0:
        print(s)
        position_fed[f'layer_{s}'] = np.hstack((pos_matrix, growing_act))
        wn_position_fed[f'layer_{s}'] = np.hstack((pos_matrix, growing_act, word_len)) 
    else:
        position_fed[f'layer_{s}'] =  np.hstack((np.array([gaussian_filter1d(row, sigma=s) for row in pos_matrix]),growing_act))
        wn_position_fed[f'layer_{s}'] = np.hstack((np.array([gaussian_filter1d(row, sigma=s) for row in pos_matrix]),growing_act, word_len))


np.savez('/data/LLMs/data_processed/fedorenko/acts/X_pos.npz', **position_fed)
np.savez('/data/LLMs/data_processed/fedorenko/acts/X_wn+pos.npz', **wn_position_fed)
#fed_position = np.load('/data/LLMs/data_processed/fedorenko/acts/X_soft+grow.npz')['layer1']
#fed_position_word_len = np.hstack((fed_position, np.expand_dims(word_len,axis=-1)))
#np.savez('/data/LLMs/data_processed/fedorenko/acts/X_soft+grow.npz')

0.0


### Blank

In [25]:
def get_pos_for_story(len_story, len_pos=30):
    return np.clip((np.arange(0,len_pos) * np.ones((len_story,len_pos))).T,
                    np.zeros(len_story), np.clip(np.arange(0,len_story),0,len_pos) ).T
    
def get_pos_for_stories(story_lens, len_pos=30):
    return np.concatenate([get_pos_for_story(l, len_pos) for l in story_lens])


dataset='blank'
blank_data = np.load(f"/data/LLMs/data_processed/{dataset}/text/story_data_dict.npz")
experiment_txt = []
data_labels = []
for key, val in blank_data.items():
    experiment_txt.extend(val)
    data_labels.extend(np.repeat(key, len(val)))

story_lens = np.diff(np.append(np.unique(data_labels, return_index=True)[1],1317))
blank_simple_acts = {}
blank_simple_acts_pos = {}

for i in np.arange(3,51):
    blank_acts_pos = get_pos_for_stories(story_lens,i) # get the ramping activations used by Antonello 
    blank_acts_pos = blank_acts_pos[:, 1:] # remove the 0th dimension since it's a constant anyways
    blank_acts = np.concatenate((np.array(([len(x.split()) for x in experiment_txt] * 1)).reshape(1,1317).T, blank_acts_pos),axis=1) # stack on a word rate feature
    blank_simple_acts[f'layer_{i}'] = blank_acts
    blank_simple_acts_pos[f"layer_{i}"] = blank_simple_acts_pos
    
np.savez('/data/LLMs/data_processed/blank/acts/X_pos-WN', **blank_simple_acts)
np.savez('/data/LLMs/data_processed/blank/acts/X_pos', **blank_simple_acts_pos)
